In [7]:
import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import scipy
from scipy.spatial import cKDTree
from scipy.spatial import Delaunay
from scipy.optimize import basinhopping
import copy
from sklearn.cluster import MiniBatchKMeans
from basic_functions import *

In [10]:

# Set initial values and parameters
start = jnp.array((1., 1., 1.))#.reshape(3, 1)
nt = 1 #transition step size
subset_size = 500 #cell number


In [12]:
#REAL trajectory
ys_fixed = Euler_jnp(params_fixed)

#CELL CENTER
Xtrain_fixed = ys_fixed.transpose()
sample = MiniBatchKMeans(n_clusters=subset_size).fit(Xtrain_fixed).cluster_centers_

{'sigma': 10, 'beta': 2.6666666666666665, 'rho': 28, 'length': 100000, 'dt': 0.005}


TypeError: JAX only supports number and bool dtypes, got dtype <class 'object'> in array

In [4]:
#vertices of barycentric simplexes
vertices = sample[Delaunay(sample).simplices]

#convert the coordinates of a point from Cartesian to Barycentric weights
#I use numpy here for now so this runs fast. But we probably need to jax when taking gradient.
def car2bar_weight(pt):
                                 
    #Cartesian coordinates
    x, y, z = pt
    vec = np.array((1,x,y,z))#vector for barycentric matrix computing
    
    for i in range(len(vertices)):
        #ri: vertices of a Barycentric simplex
        r1, r2, r3, r4 = vertices[i]

        x1, y1, z1 = r1
        x2, y2, z2 = r2
        x3, y3, z3 = r3
        x4, y4, z4 = r4
    
        T = np.array(([1., 1., 1., 1.],
                      [x1, x2, x3, x4],
                      [y1, y2, y3, y4],
                      [z1, z2, z3, z4]))
        T_inverse = np.linalg.inv(T)
        bar = T_inverse.dot(vec)

        if np.min(bar)>= 0.:#to make sure the point is inside or on the simplex
            return bar
        
